In [ ]:
import math
import torch
from collections import OrderedDict
from torch.nn import utils, functional as F
from torch.optim import Adam
from torch.backends import cudnn
from torchvision import transforms
from dssnet import build_model, weights_init
from loss import Loss
from tools.visual import Viz_visdom,plot_image, make_simple_grid
from torch.autograd import Variable
from torchvision.utils import save_image
from solver import Solver

In [ ]:
    def test(self, num, use_crf=False):
        if use_crf: from tools.crf_process import crf
        avg_mae, img_num = 0.0, len(self.test_dataset)
        avg_prec, avg_recall = torch.zeros(num), torch.zeros(num)
        with torch.no_grad():
            for i,data in enumerate(self.test_dataset): #(img, labels) in enumerate(self.test_dataset):
                images,labels = data['image'], data['label']
                images = images.type(torch.cuda. FloatTensor)
                labels= labels.type(torch.cuda.FloatTensor)
                #images = self.transform(img).unsqueeze(0)
                #labels = labels.unsqueeze(0)
                shape = labels.size()[2:]
                #print(shape)
                images = images.to(self.device)
                labels=labels.to(self.device)
                prob_pred = self.net(images)

                prob_pred = torch.mean(torch.cat([prob_pred[i] for i in self.select], dim=1), dim=1, keepdim=True)
                prob_pred = F.interpolate(prob_pred, size=shape, mode='bilinear', align_corners=True).cpu().data
                print(prob_pred[0].size())
                result_dir='C:/Users/Paul Vincent Nonat/Documents/Graduate Student Files/results/'
                save_image(prob_pred[0],result_dir+'result'+str(i)+'.png')
                if use_crf:
                    prob_pred = crf(img, prob_pred.numpy(), to_tensor=True)
                mae = self.eval_mae(prob_pred, labels)
                prec, recall = self.eval_pr(prob_pred, labels, num)
                print(num)
                print("[%d] mae: %.4f" % (i, mae))
                print("[%d] mae: %.4f" % (i, mae), file=self.test_output)
                avg_mae += mae
                avg_prec, avg_recall = avg_prec + prec, avg_recall + recall
        avg_mae, avg_prec, avg_recall = avg_mae / img_num, avg_prec / img_num, avg_recall / img_num
        score = (1 + self.beta ** 2) * avg_prec * avg_recall / (self.beta ** 2 * avg_prec + avg_recall)
        score[score != score] = 0  # delete the nan
        print('average mae: %.4f, max fmeasure: %.4f' % (avg_mae, score.max()))
        print('average mae: %.4f, max fmeasure: %.4f' % (avg_mae, score.max()), file=self.test_output)

In [ ]:
data_dir =   'C:/Users/paulvincentnonat/Documents/GitHub/Saliency_Dataset/DUTS/'
tra_image_dir = 'DUTS-TR/DUTS-TR-Image/'
tra_label_dir = 'DUTS-TR/DUTS-TR-Mask/'
test_image_dir = 'DUTS-TE/DUTS-TE-Image/'
test_label_dir = 'DUTS-TE/DUTS-TE-Mask/'
enableInpaintAug = False
batch_size_train=32
batch_size_val=1
image_ext = '.jpg'
label_ext = '.png'

In [ ]:
tra_img_name_list = glob.glob(data_dir + tra_image_dir + '*' + image_ext)
print("data_dir + tra_image_dir + '*' + image_ext: ", data_dir + tra_image_dir + '*' + image_ext)
test_img_name_list = glob.glob(data_dir + test_image_dir + '*' + image_ext)
print("data_dir + test_image_dir + '*' + image_ext: ", data_dir + test_image_dir + '*' + image_ext)

tra_lbl_name_list = []
for img_path in tra_img_name_list:
    img_name = img_path.split("\\")[-1]
    aaa = img_name.split(".")
    bbb = aaa[0:-1]
    imidx = bbb[0]
    for i in range(1,len(bbb)):
        imidx = imidx + "." + bbb[i]
    tra_lbl_name_list.append(data_dir + tra_label_dir + imidx + label_ext)

test_lbl_name_list = []
for img_path in test_img_name_list:
    img_name = img_path.split("\\")[-1]
    aaa = img_name.split(".")
    bbb = aaa[0:-1]
    imidx = bbb[0]
    for i in range(1,len(bbb)):
        imidx = imidx + "." + bbb[i]
    test_lbl_name_list.append(data_dir + test_label_dir + imidx + label_ext)

print("---")
print("train images: ", len(tra_img_name_list))
print("train labels: ", len(tra_lbl_name_list))
print("---")


print("---")
print("test images: ", len(test_img_name_list))
print("test labels: ", len(test_lbl_name_list))
print("---")

train_num = len(tra_img_name_list)
test_num = len(test_img_name_list)

salobj_dataset = SalObjDataset(
    img_name_list=tra_img_name_list,
    lbl_name_list=tra_lbl_name_list,
    transform=transforms.Compose([
        RescaleT(256),
        RandomCrop(224),
        ToTensorLab(flag=0)]),
        category="train",
        enableInpaintAug=enableInpaintAug)
salobj_dataset_test = SalObjDataset(
    img_name_list=test_img_name_list,
    lbl_name_list=test_lbl_name_list,
    transform=transforms.Compose([
        RescaleT(256),
        RandomCrop(224),
        ToTensorLab(flag=0)]),
        category="test",
        enableInpaintAug=enableInpaintAug)

In [ ]:
test_loader = DataLoader(salobj_dataset_test, batch_size=batch_size_val, shuffle=True, num_workers=1)
if not os.path.exists(config.test_fold): os.mkdir(config.test_fold)
test = Solver(None, None, test_loader, config)
test.test(100, use_crf=config.use_crf)